# Torch bridges examples

In [9]:
import copy

import numpy as np
from datasets import load_dataset
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)
from plaid_ops.mesh.feature_engineering import update_dataset_with_sdf
from plaid_ops.mesh.transformations import (
    compute_bounding_box,
    project_on_regular_grid,
)
from torch.utils.data import DataLoader

from plaid_bridges.common import BaseRegressionDataset
from plaid_bridges.torch import GridFieldsAndScalarsOfflineTransformer

## Projection on constant rectilinear grid, with scalars as constant fields

In [10]:
hf_dataset = load_dataset(
    "PLAID-datasets/2D_Multiscale_Hyperelasticity", split="all_samples"
)
pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids_train = pb_def.get_split("DOE_train")[:10]

dataset_train, _ = huggingface_dataset_to_plaid(
    hf_dataset, ids=ids_train, processes_number=5
)
n_train = len(dataset_train)
print(dataset_train)

Converting Hugging Face dataset to plaid dataset...


100%|██████████| 10/10 [00:00<00:00, 268.38it/s]

Dataset(10 samples, 4 scalars, 0 time_series, 7 fields)


In [11]:
dims = (101, 101)
dataset_train = update_dataset_with_sdf(dataset_train, verbose=True)

print(dataset_train.get_sample_ids())

bbox = compute_bounding_box(dataset_train)
projected_dataset_train = project_on_regular_grid(
    dataset_train, dimensions=dims, bbox=bbox, verbose=True
)

all_feat_ids = dataset_train[ids_train[0]].get_all_features_identifiers()
scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values()]

100%|██████████| 10/10 [00:00<00:00, 14.94it/s]


[376, 377, 378, 379, 380, 381, 382, 383, 384, 385]


100%|██████████| 10/10 [00:00<00:00, 33.58it/s]


In [12]:
offline_in_transformer = GridFieldsAndScalarsOfflineTransformer(
    dimensions=dims,
    features_identifiers=[field_features[0], scalar_features[0]],
)
offline_out_transformer = GridFieldsAndScalarsOfflineTransformer(
    dimensions=dims,
    features_identifiers=[scalar_features[1], field_features[1]],
)

In [13]:
torch_dataset = BaseRegressionDataset(
    dataset=projected_dataset_train,
    offline_in_transformer=offline_in_transformer,
    offline_out_transformer=offline_out_transformer,
)

In [14]:
loader = DataLoader(
    torch_dataset,
    batch_size=2,
    shuffle=False,
)

out_feat_id = scalar_features[0]

before = copy.deepcopy(
    projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
)

predictions = []
for batch_x, batch_y in loader:
    for torch_sample in batch_y:
        predictions.append(torch_sample.detach().cpu())

pred_projected_dataset_train = offline_out_transformer.inverse_transform(
    projected_dataset_train, predictions
)

after = copy.deepcopy(
    pred_projected_dataset_train[ids_train[1]].get_feature_from_identifier(out_feat_id)
)

print("Error after transform then inverse transform (2nd sample):")
print(np.linalg.norm(after - before) / np.linalg.norm(before))

Error after transform then inverse transform (2nd sample):
0.0


In [15]:
print(torch_dataset)

RegressionDataset, initialized from plaid:Dataset(10 samples, 4 scalars, 0 time_series, 8 fields)


In [16]:
print(offline_out_transformer)
print(offline_out_transformer.show_details())

Transformer (2 input features)
Transformer: features=['C12 (scalar)', 'psi (field)']
